<a href="https://colab.research.google.com/github/xascendent/tic-tac-toe/blob/main/Copy_of_Evaluating_RAG_with_Ragas_(2025)_AI_Makerspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [19]:
!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9

In [20]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.0 MB/s eta 0:00:00
   ━━━━

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [10]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your OpenAI API key!··········


OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [4]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

Please enter your Ragas API key!··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [11]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [12]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0   171k      0 --:--:-- --:--:-- --:--:--  171k


In [13]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   466k      0 --:--:-- --:--:-- --:--:--  469k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [21]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [22]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [23]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [24]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is Gooogle's role in the development of LLMs?,[Code may be the best application The ethics o...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,How LLMs handle Spanish grammar?,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,"So like, what did we figure out about AI in 20...",[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Large Language Models (LLMs) were rec...",single_hop_specifc_query_synthesizer
3,what openai do in 2023 and how it affect ai de...,[easy to follow. The rest of the document incl...,"In 2023, OpenAI was a significant topic of dis...",single_hop_specifc_query_synthesizer
4,How have automated evaluations for LLMs become...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
5,How have automated evaluations for LLMs become...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
6,How have automated evaluations for LLMs become...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
7,How has increased competition and universal ac...,[<1-hop>\n\nPrompt driven app generation is a ...,Increased competition among AI model providers...,multi_hop_abstract_query_synthesizer
8,How has the development of Claude 3 and the ri...,[<1-hop>\n\ndependent on AGI itself. A model t...,The development of Claude 3 and the rise of in...,multi_hop_specific_query_synthesizer
9,How did the advancements in GPT-4 and its succ...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, significant advancements in GPT-4 and...",multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [25]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/b8998a67-1dc2-4f93-bc68-1cc50170fbcf


'https://app.ragas.io/dashboard/alignment/testset/b8998a67-1dc2-4f93-bc68-1cc50170fbcf'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [26]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

#### ❓ Question:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?
A- ensures that adjacent text chunks overlap, preserving the continuity of information in the chunked data. It also helps prevent document fragmentation by carefully managing where the text is split.  overlap prevents lost of context.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [28]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [29]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [30]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [31]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [32]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [33]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [35]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [36]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [37]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [38]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [39]:
response["response"]

'LLM agents are useful in several ways:\n\n1. **Ease of Building**: They can be constructed with relatively simple code (a few hundred lines of Python), making the technology more accessible than previously thought. While building a high-quality model requires significant training data and computational resources, it is no longer solely in the domain of the super-rich.\n\n2. **Running on Personal Devices**: Recent advancements have made it possible to run LLMs on personal computers, even those with lower specifications. This democratizes access to LLM technology.\n\n3. **Code Generation**: LLMs excel at writing code, which is one of their most notable capabilities. They can generate code, test it, and refine it based on error messages, making them effective tools for software development.\n\n4. **Assistance in Tasks**: They can act as agents that perform tasks on behalf of users, although this application is still in the prototype stage and faces challenges related to credibility and e

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [40]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [41]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht is Gooogle's role in the development of LLMs?,[So training an LLM still isn’t something a ho...,[Code may be the best application The ethics o...,Google has played a significant role in the de...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,How LLMs handle Spanish grammar?,[So training an LLM still isn’t something a ho...,[Based Development As a computer scientist and...,The provided context does not specifically add...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,"So like, what did we figure out about AI in 20...",[Gullibility is the biggest unsolved problem\n...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, we made significant strides in unders...","In 2023, Large Language Models (LLMs) were rec...",single_hop_specifc_query_synthesizer
3,what openai do in 2023 and how it affect ai de...,[The biggest innovation here is that it opens ...,[easy to follow. The rest of the document incl...,"In 2023, OpenAI made significant advancements ...","In 2023, OpenAI was a significant topic of dis...",single_hop_specifc_query_synthesizer
4,How have automated evaluations for LLMs become...,"[Still, I’m surprised that no-one has beaten t...",[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for Large Language Model...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
5,How have automated evaluations for LLMs become...,[It’s become abundantly clear over the course ...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for LLMs (Large Language...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
6,How have automated evaluations for LLMs become...,[The rise of inference-scaling “reasoning” mod...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations for large language model...,Automated evaluations for LLMs have become cru...,multi_hop_abstract_query_synthesizer
7,How has increased competition and universal ac...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nPrompt driven app generation is a ...,Increased competition and universal access to ...,Increased competition among AI model providers...,multi_hop_abstract_query_synthesizer
8,How has the development of Claude 3 and the ri...,[The rise of inference-scaling “reasoning” mod...,[<1-hop>\n\ndependent on AGI itself. A model t...,The development of Claude 3 and the rise of in...,The development of Claude 3 and the rise of in...,multi_hop_specific_query_synthesizer
9,How did the advancements in GPT-4 and its succ...,[This increase in efficiency and reduction in ...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The advancements in GPT-4 and its successors i...,"In 2024, significant advancements in GPT-4 and...",multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [42]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [43]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [44]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[24]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMKGHy on tokens per min (TPM): Limit 30000, Used 28893, Requested 2372. Please try again in 2.53s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMKGHy on tokens per min (TPM): Limit 30000, Used 29848, Requested 2174. Please try again in 4.044s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[7]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMKGH

{'context_recall': 0.6361, 'faithfulness': 0.7143, 'factual_correctness': 0.4200, 'answer_relevancy': 0.8542, 'context_entity_recall': 0.3906, 'noise_sensitivity_relevant': 0.2740}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [46]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [47]:
!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [48]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [49]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [50]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [51]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents are viewed as potentially useful for specific tasks, particularly in writing code. The context highlights that LLMs are effective at coding due to the simpler grammar rules of programming languages compared to natural languages. However, there is skepticism about their overall utility, primarily due to issues of gullibility, where LLMs may accept and propagate false information. This raises concerns about their reliability in making meaningful decisions or acting on behalf of users. Despite the excitement around AI agents, there are few production examples, and the challenges of misinformation and gullibility remain significant obstacles to realizing their full potential.'

In [52]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [53]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[24]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMKGHy on tokens per min (TPM): Limit 30000, Used 29312, Requested 2372. Please try again in 3.368s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[13]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMKGHy on tokens per min (TPM): Limit 30000, Used 29056, Requested 2583. Please try again in 3.278s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-mHCkECq8EjzzYH6rxhkMK

{'context_recall': 0.7250, 'faithfulness': 0.7786, 'factual_correctness': 0.4292, 'answer_relevancy': 0.8554, 'context_entity_recall': 0.4125, 'noise_sensitivity_relevant': 0.2981}

#### ❓ Question:

Which system performed better, on what metrics, and why?
Overall Cohere’s Rerank model performed better with the expection of Noise Sensitivity.  Cohere was much better at context recall, and faithfulnes (responses that align more closely with retrieved content).